In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [6]:
from pyspark.sql.dataframe import DataFrame
DataFrame.old_show = getattr(DataFrame, 'old_show', DataFrame.show)
def show(self, n=20, truncate=20, vertical=False):
    self.persist()
    print(self.count())
    self.old_show(n, truncate=truncate, vertical=vertical)
    return self

DataFrame.show = show

import random
def sql(self, q):
    name = 'table_{}'.format(random.randint(0000, 9999))
    self.createOrReplaceTempView(name)
    q = q.format(T=name)
    print(q)
    return df.sql_ctx.sql(q)
DataFrame.sql = sql


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StringType, BooleanType
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate()





In [131]:
df = spark.read.json('./jsons/20_04_09__21_50_08.json')

In [129]:
df.sh

DataFrame[address: string, area: string, area_kitchen: string, area_live: string, categoryId: bigint, categorySlug: string, commission: string, commission_amount: string, datetm: string, deposit: string, dynx_category: string, dynx_pagetype: string, dynx_price: bigint, dynx_prodid: bigint, dynx_region: string, dynx_user: string, dynx_vertical: bigint, floor: string, floors_count: string, house_type: string, isASDClient: bigint, isShop: bigint, itemID: bigint, itemPrice: bigint, kol_vo_krovatey: string, kol_vo_spalnykh_mest: string, lease_period: string, locationId: bigint, microCategoryId: bigint, offer_type: string, offline_check_result: string, old_prise: string, pageType: string, posted_datetm: string, rooms: string, seller_name: string, seller_url: string, sub_price: string, text: string, title: string, url: string, vertical: string, views: string, withDelivery: bigint]

In [7]:
df.select('address', 'area', 'area_kitchen', 'area_live', 'categoryId', 'categorySlug', 'commission', 'commission_amount', 'datetm','deposit', 'dynx_category', 'dynx_pagetype', 'dynx_price', 'dynx_prodid', 'dynx_region', 'dynx_user', 'dynx_vertical',).show(100)

114
+--------------------+-------+------------+---------+----------+------------+-----------+-----------------+--------------------+--------+-------------+-------------+----------+-----------+--------------------+---------+-------------+
|             address|   area|area_kitchen|area_live|categoryId|categorySlug| commission|commission_amount|              datetm| deposit|dynx_category|dynx_pagetype|dynx_price|dynx_prodid|         dynx_region|dynx_user|dynx_vertical|
+--------------------+-------+------------+---------+----------+------------+-----------+-----------------+--------------------+--------+-------------+-------------+----------+-----------+--------------------+---------+-------------+
|Санкт-Петербург, ...|  35 м²|        6 м²|    25 м²|        24|    kvartiry|       null|             null|2020-03-30 18:31:...|    null|     kvartiry|         item|      1800| 1766646931|     sankt-peterburg|        b|            1|
|Санкт-Петербург, ...|  25 м²|        null|     null|       

DataFrame[address: string, area: string, area_kitchen: string, area_live: string, categoryId: bigint, categorySlug: string, commission: string, commission_amount: string, datetm: string, deposit: string, dynx_category: string, dynx_pagetype: string, dynx_price: bigint, dynx_prodid: bigint, dynx_region: string, dynx_user: string, dynx_vertical: bigint]

In [132]:
from datetime import timedelta, date, datetime
from pyspark.sql.types import TimestampType
import re

months = {m: i for i, m in enumerate('января февраля марта апреля мая июня июля августа сентября октября ноября декабря'.split(), 1)}
def p(s):
    try:
        if 'сегодня' in s:
            result = datetime.now()
        elif 'вчера' in s:
            result = datetime.now() - timedelta(1)
        else:
            found = re.search('(\d+) (\w+) в', s)
            result = datetime.now().replace(month=int(months[found.group(2)]), day=int(found.group(1)))

        found = re.search('(\d\d):(\d\d)', s)
        hour, minute = found.groups()
        result = result.replace(hour=int(hour), minute=int(minute), second=0, microsecond=0)

        return result
    except Exception as e:
        print(f'Exception {e} while converting "{s}"')
        return None

In [133]:
print(p('сегодня в 20:41 '))
print(p('вчера в 20:41 '))
print(p('4 апреля в 20:33'))
print(p('30 марта в 13:08'))


2020-04-09 20:41:00
2020-04-08 20:41:00
2020-04-04 20:33:00
2020-03-30 13:08:00


In [141]:
df.drop('dynx_user', 'dynx_vertical', 'vertical', 'withDelivery', 'dynx_category', 'dynx_pagetype', 'dynx_category', 'categorySlug', 'categoryId', 'dynx_price', 'pageType', 'isASDClient', 'dynx_prodid') \
    .drop('locationId', 'microCategoryId', 'dynx_region') \
    .withColumnRenamed('kol_vo_krovatey', 'beds') \
    .withColumnRenamed('kol_vo_spalnykh_mest', 'sleep_places') \
    .withColumnRenamed('floors_count', 'floors') \
    .withColumnRenamed('commission_amount', 'comm_amt') \
    .withColumnRenamed('commission', 'comm') \
    .where('offer_type="Сдам" and lease_period="На длительный срок"') \
    .drop('offer_type', 'lease_period') \
    .select('posted_datetm', 'url', 'area', 'area_kitchen', 'area_live', 'itemPrice') \
    .withColumn('posted', F.udf(p, TimestampType())('posted_datetm')) \
    .withColumn('area', F.regexp_extract(F.col('area'), '(\d+)', 1).cast(IntegerType())) \
    .withColumn('area_kitchen', F.regexp_extract(F.col('area_kitchen'), '(\d+)', 1).cast(IntegerType())) \
    .withColumn('area_live', F.regexp_extract(F.col('area_live'), '(\d+)', 1).cast(IntegerType())) \
    .withColumn('itemPrice', F.col('itemPrice').cast(IntegerType())) \
    .sort(F.asc('posted')) \
    .show(20, False)

2065
+----------------+--------------------------------------------------------------------------------------+----+------------+---------+---------+-------------------+
|posted_datetm   |url                                                                                   |area|area_kitchen|area_live|itemPrice|posted             |
+----------------+--------------------------------------------------------------------------------------+----+------------+---------+---------+-------------------+
|3 апреля в 10:04|https://www.avito.ru//sankt-peterburg/kvartiry/2-k_kvartira_50_m_16_et._1898727675    |50  |28          |22       |39000    |2020-04-03 10:04:00|
|3 апреля в 11:16|https://www.avito.ru//sankt-peterburg/kvartiry/2-k_kvartira_49_m_69_et._1215318857    |49  |9           |32       |26000    |2020-04-03 11:16:00|
|3 апреля в 11:55|https://www.avito.ru//sankt-peterburg/kvartiry/studiya_25_m_922_et._1333739731        |25  |null        |null     |20000    |2020-04-03 11:55:00|
|3 апреля в

2065

In [130]:
for col in df.columns:
    df.groupby(col).count().sort('count', ascending=False).show(5)

3356
+--------------------+-----+
|             address|count|
+--------------------+-----+
|Санкт-Петербург, ...|   38|
|Санкт-Петербург, ...|   35|
|Санкт-Петербург, ...|   35|
|Санкт-Петербург, ...|   26|
|Санкт-Петербург, ...|   25|
+--------------------+-----+
only showing top 5 rows

308
+-----+-----+
| area|count|
+-----+-----+
|40 м²|  366|
|30 м²|  270|
|25 м²|  228|
|45 м²|  223|
|35 м²|  202|
+-----+-----+
only showing top 5 rows

126
+------------+-----+
|area_kitchen|count|
+------------+-----+
|        null| 1460|
|       10 м²|  583|
|       12 м²|  446|
|        9 м²|  336|
|        8 м²|  330|
+------------+-----+
only showing top 5 rows

195
+---------+-----+
|area_live|count|
+---------+-----+
|     null| 1395|
|    18 м²|  577|
|    20 м²|  446|
|    17 м²|  302|
|    16 м²|  258|
+---------+-----+
only showing top 5 rows

1
+----------+-----+
|categoryId|count|
+----------+-----+
|        24| 5682|
+----------+-----+

1
+------------+-----+
|categorySlug|count|
+--

61
+--------------------+------+------------+---------+----------+------------+-----------+-----------------+--------------------+-------+-------------+-------------+----------+-----------+---------------+-----+------------+----------+-----------+------+----------+---------+---------------+--------------------+------------------+----------+---------------+----------+----------------+--------+----------------+------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+------------+
|             address|  area|area_kitchen|area_live|categoryId|categorySlug| commission|commission_amount|              datetm|deposit|dynx_category|dynx_pagetype|dynx_price|dynx_prodid|    dynx_region|floor|floors_count|house_type|isASDClient|isShop|    itemID|itemPrice|kol_vo_krovatey|kol_vo_spalnykh_mest|      lease_period|locationId|microCategoryId|offer_type|       old_prise|pageType|   posted_datetm| rooms|       seller

DataFrame[address: string, area: string, area_kitchen: string, area_live: string, categoryId: bigint, categorySlug: string, commission: string, commission_amount: string, datetm: string, deposit: string, dynx_category: string, dynx_pagetype: string, dynx_price: bigint, dynx_prodid: bigint, dynx_region: string, floor: string, floors_count: string, house_type: string, isASDClient: bigint, isShop: bigint, itemID: bigint, itemPrice: bigint, kol_vo_krovatey: string, kol_vo_spalnykh_mest: string, lease_period: string, locationId: bigint, microCategoryId: bigint, offer_type: string, old_prise: string, pageType: string, posted_datetm: string, rooms: string, seller_name: string, seller_url: string, sub_price: string, text: string, title: string, url: string, views: string, withDelivery: bigint]

In [128]:
df.select('address', 'area', 'area_kitchen', 'area_live', 'categoryId', 'categorySlug', 'commission', 'commission_amount', 'datetm','deposit', 'dynx_category', 'dynx_pagetype', 'dynx_price', 'dynx_prodid', 'dynx_region', 'dynx_user', 'dynx_vertical',).show(100)

5682
+--------------------+-------+------------+---------+----------+------------+-----------+-----------------+--------------------+--------+-------------+-------------+----------+-----------+--------------------+---------+-------------+
|             address|   area|area_kitchen|area_live|categoryId|categorySlug| commission|commission_amount|              datetm| deposit|dynx_category|dynx_pagetype|dynx_price|dynx_prodid|         dynx_region|dynx_user|dynx_vertical|
+--------------------+-------+------------+---------+----------+------------+-----------+-----------------+--------------------+--------+-------------+-------------+----------+-----------+--------------------+---------+-------------+
|Санкт-Петербург, ...|  36 м²|        null|    20 м²|        24|    kvartiry|  Посредник|             60 %|2020-04-06 00:19:...|0.5 мес.|     kvartiry|         item|     18000|  953418539|     sankt-peterburg|        b|            1|
|Санкт-Петербург, ...|  30 м²|        null|     null|      

DataFrame[address: string, area: string, area_kitchen: string, area_live: string, categoryId: bigint, categorySlug: string, commission: string, commission_amount: string, datetm: string, deposit: string, dynx_category: string, dynx_pagetype: string, dynx_price: bigint, dynx_prodid: bigint, dynx_region: string, dynx_user: string, dynx_vertical: bigint]

In [18]:
df.select('floor', 'floors_count', 'house_type', 'isASDClient', 'isShop', 'itemID', 'itemPrice', 'kol_vo_krovatey', 'kol_vo_spalnykh_mest','korpus', 'korpus_stroenie', 'lease_period', 'locationId', 'microCategoryId', 'nazvanie_novostroyki', 'nazvanie_obekta_nedvizhimosti').show(100)

58
+-----+------------+----------+-----------+------+----------+---------+---------------+--------------------+------+-----------------+------------------+----------+---------------+--------------------+-----------------------------+
|floor|floors_count|house_type|isASDClient|isShop|    itemID|itemPrice|kol_vo_krovatey|kol_vo_spalnykh_mest|korpus|  korpus_stroenie|      lease_period|locationId|microCategoryId|nazvanie_novostroyki|nazvanie_obekta_nedvizhimosti|
+-----+------------+----------+-----------+------+----------+---------+---------------+--------------------+------+-----------------+------------------+----------+---------------+--------------------+-----------------------------+
|    7|          16| Кирпичный|          0|     0|1844735206| 14470000|           null|                null|  null|             null|              null|    653240|           4920|                null|                         null|
|   16|          19|Монолитный|          0|     0|1901762309|    24000|  

DataFrame[floor: string, floors_count: string, house_type: string, isASDClient: bigint, isShop: bigint, itemID: bigint, itemPrice: bigint, kol_vo_krovatey: string, kol_vo_spalnykh_mest: string, korpus: bigint, korpus_stroenie: string, lease_period: string, locationId: bigint, microCategoryId: bigint, nazvanie_novostroyki: bigint, nazvanie_obekta_nedvizhimosti: string]

In [25]:
df.select('nomer_kvartiry', 'offer_type', 'ofitsialnyy_zastroyshchik', 'old_prise', 'otdelka', 'pageType', 'posted_datetm', 'proektnaya_deklaratsiya','rooms', 'seller_id', 'seller_name', 'seller_url', 'sub_price', 'text', 'tip_uchastiya', 'title', 'type', 'url', 'vertical', 'views', 'withDelivery').show(100)

58
+--------------+----------+-------------------------+----------------+--------+--------+----------------+-----------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------+--------------------+--------+-----------+------------+
|nomer_kvartiry|offer_type|ofitsialnyy_zastroyshchik|       old_prise| otdelka|pageType|   posted_datetm|proektnaya_deklaratsiya| rooms|           seller_id|         seller_name|          seller_url|           sub_price|                text|tip_uchastiya|               title|       type|                 url|vertical|      views|withDelivery|
+--------------+----------+-------------------------+----------------+--------+--------+----------------+-----------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------+--------------------+

DataFrame[nomer_kvartiry: string, offer_type: string, ofitsialnyy_zastroyshchik: string, old_prise: string, otdelka: string, pageType: string, posted_datetm: string, proektnaya_deklaratsiya: string, rooms: string, seller_id: string, seller_name: string, seller_url: string, sub_price: string, text: string, tip_uchastiya: bigint, title: string, type: string, url: string, vertical: string, views: string, withDelivery: bigint]

In [16]:
df.select('seller_id').show()

58
+--------------------+
|           seller_id|
+--------------------+
|                user|
|                user|
|                user|
|mkelite?gdlkerfdn...|
|                user|
|                user|
|                user|
|mirkvru?gdlkerfdn...|
|                user|
|                user|
|                user|
|                user|
|psk-info?gdlkerfd...|
|i178394876?gdlker...|
|cds-spb-ru?gdlker...|
|landrin-dom?gdlke...|
|psk-info?gdlkerfd...|
|                user|
|                user|
|                user|
+--------------------+
only showing top 20 rows



DataFrame[seller_id: string]

In [6]:
# total products per shop
df.groupBy("shop").pivot("date").agg(F.count('product_id')).show()


4
+-------+--------+--------+--------+--------+--------+
|   shop|20-03-16|20-03-17|20-03-18|20-03-19|20-03-21|
+-------+--------+--------+--------+--------+--------+
| prisma|    8991|    8689|    8629|    8660|    8680|
|  metro|   19599|   19645|   19365|   19702|   19727|
|karusel|   12501|   13139|   12450|   12642|   12803|
|  lenta|   14394|   14176|   13748|   14087|   14041|
+-------+--------+--------+--------+--------+--------+



DataFrame[shop: string, 20-03-16: bigint, 20-03-17: bigint, 20-03-18: bigint, 20-03-19: bigint, 20-03-21: bigint]

In [7]:
# total products with discounts per shop 
df.where('old_price is not null').groupBy("shop").pivot("date").agg(F.count('product_id')).show()


4
+-------+--------+--------+--------+--------+--------+
|   shop|20-03-16|20-03-17|20-03-18|20-03-19|20-03-21|
+-------+--------+--------+--------+--------+--------+
| prisma|     997|     934|     908|     783|     759|
|  metro|    2707|    2813|    2762|    2833|    2988|
|karusel|    1836|    2022|    1629|    1141|    1802|
|  lenta|    1513|    1531|    1490|    1430|    1305|
+-------+--------+--------+--------+--------+--------+



DataFrame[shop: string, 20-03-16: bigint, 20-03-17: bigint, 20-03-18: bigint, 20-03-19: bigint, 20-03-21: bigint]

In [8]:
# total average price per shop 
df.groupBy("shop").pivot("date").agg(F.sum('price')/F.count('product_id')).show()


4
+-------+------------------+------------------+------------------+------------------+------------------+
|   shop|          20-03-16|          20-03-17|          20-03-18|          20-03-19|          20-03-21|
+-------+------------------+------------------+------------------+------------------+------------------+
| prisma|188.10001347850155|188.34344018157324|189.03996892411402|192.07068617674275|192.57394608264696|
|  metro|  291.260432704589| 287.4751096981541| 289.6296090938372| 289.3170404041156|284.33816950983623|
|karusel|163.46044942153262|162.95179831070737|166.14586250631686| 168.7916300962232|166.81134088776088|
|  lenta|212.23314121221028|210.92874393766877|213.27697006009296|213.59038444901407| 213.3129290605269|
+-------+------------------+------------------+------------------+------------------+------------------+



DataFrame[shop: string, 20-03-16: double, 20-03-17: double, 20-03-18: double, 20-03-19: double, 20-03-21: double]